In [2]:
import pandas as pd
import string
from nltk.corpus import stopwords
import re
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer, PorterStemmer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
#from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

## **Example 2**

In [5]:
news_df = pd.read_csv('C:/Users/intelUS-Economic-News.csv', encoding = 'ISO-8859-1')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/intelUS-Economic-News.csv'

In [50]:
news_df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [51]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               8000 non-null   int64  
 1   _golden                8000 non-null   bool   
 2   _unit_state            8000 non-null   object 
 3   _trusted_judgments     8000 non-null   int64  
 4   _last_judgment_at      8000 non-null   object 
 5   positivity             1420 non-null   float64
 6   positivity:confidence  3775 non-null   float64
 7   relevance              8000 non-null   object 
 8   relevance:confidence   8000 non-null   float64
 9   articleid              8000 non-null   object 
 10  date                   8000 non-null   object 
 11  headline               8000 non-null   object 
 12  positivity_gold        0 non-null      float64
 13  relevance_gold         0 non-null      float64
 14  text                   8000 non-null   object 
dtypes: b

In [52]:
news_df.isnull().sum()

_unit_id                    0
_golden                     0
_unit_state                 0
_trusted_judgments          0
_last_judgment_at           0
positivity               6580
positivity:confidence    4225
relevance                   0
relevance:confidence        0
articleid                   0
date                        0
headline                    0
positivity_gold          8000
relevance_gold           8000
text                        0
dtype: int64

In [53]:
news_df = news_df[['text', 'relevance']]

In [54]:
news_df.head()

,text,relevance
0,NEW YORK -- Yields on most certificates of dep...,yes
1,The Wall Street Journal Online</br></br>The Mo...,no
2,WASHINGTON -- In an effort to achieve banking ...,no
3,The statistics on the enormous costs of employ...,no
4,NEW YORK -- Indecision marked the dollar's ton...,yes


In [55]:
news_df['relevance'].value_counts()

no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [56]:
news_df['relevance'] = news_df.relevance.map({'yes': 1, 'no':0 , 'not sure': 0})

In [57]:
news_df = news_df[['text', 'relevance']]

In [ ]:
news_df.head()

In [ ]:
news_df['relevance'].value_counts()

In [60]:
x = news_df['text']

In [61]:
x.head()

0    NEW YORK -- Yields on most certificates of dep...
1    The Wall Street Journal Online</br></br>The Mo...
2    WASHINGTON -- In an effort to achieve banking ...
3    The statistics on the enormous costs of employ...
4    NEW YORK -- Indecision marked the dollar's ton...
Name: text, dtype: object

In [62]:
y = news_df['relevance']

In [63]:
y.head()

0    1
1    0
2    0
3    0
4    1
Name: relevance, dtype: int64

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42)

In [65]:
# function to convert the text to lower case
def convert_to_lowercase(text):
    return text.str.lower()

In [66]:
# function to remove punctuations from the text
def remove_punctuations(text):
    eng_punctuation = string.punctuation
    translator = str.maketrans('','', eng_punctuation)
    return str(text).translate(translator)

In [ ]:
# function to remove stopwords from the text
def remove_stopwords(text):
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words('english'))
    return " ".join([word for word in str(text).split() if word not in stopwords])

In [ ]:
# function to remove repeating characters
def remove_repeating_characters(text):
    return re.sub(r'(.)1+', r'1', text)

In [ ]:
# function to remove numeric text
def remove_numeric(text):
    return re.sub('[0-9]+', '', text)

In [ ]:
# Tokenizing the text
def tokenize_text(text):
    tokenizer = RegexpTokenizer('\w+')
    text = text.apply(tokenizer.tokenize)
    return text

In [ ]:
# lemmatizing the text. Converting some of the words to their root form.
def text_lematization(text):
    lm = WordNetLemmatizer()
    text = [lm.lemmatize(word) for word in text]
    return text

In [ ]:
def preprocess(text):
    text = convert_to_lowercase(text)
    text = text.apply(lambda x : remove_punctuations(x))
    text = text.apply(lambda x : remove_stopwords(x))
    text = text.apply(lambda x : remove_repeating_characters(x))
    text = text.apply(lambda x : remove_numeric(x))
    text = tokenize_text(text)
    text = text.apply(lambda x : text_lematization(x))
    text = text.apply(lambda x: " ".join(x))
    return text

In [ ]:
x_train = preprocess(x_train)

In [ ]:
x_test = preprocess(x_test)

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)

In [ ]:
x_train = vectorizer.transform(x_train)

In [ ]:
x_test = vectorizer.transform(x_test)

In [40]:
#lr model
lr_model = LogisticRegression()
#fit
lr_model.fit(x_train, y_train)

ValueError: ignored

In [41]:
#pred
lr_pred = lr_model.predict(x_test)

NotFittedError: ignored

In [ ]:
print(classification_report(y_test, lr_pred))

In [ ]:
train_lr_pred = lr_model.predict(x_train)

In [ ]:
print(classification_report(y_train, train_lr_pred))

In [ ]:
#knn
knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)

In [ ]:
knn_pred = knn_model.predict(x_test)

In [ ]:
print(classification_report(y_test, knn_pred))

In [ ]:
train_knn_pred = knn_model.predict(x_train)
print(classification_report(y_train, train_knn_pred))

In [47]:
x_train_array = x_train.toarray()

AttributeError: ignored

In [ ]:
x_test_array = x_test.toarray()

In [ ]:
nb_model = GaussianNB()
nb_model.fit(x_train_array, y_train)
nb_pred = nb_model.predict(x_test_array)

In [46]:
print(classification_report(y_test, nb_pred))

NameError: ignored

In [ ]:
train_nb_pred = nb_model.predict(x_train_array)
print(classification_report(y_train, train_nb_pred))

In [45]:
svm_model = SVC()
svm_model.fit(x_train, y_train)

ValueError: ignored

In [ ]:
svm_pred = svm_model.predict(x_test)

In [ ]:
print(classification_report(y_test, svm_pred))

In [44]:
train_svm_pred = svm_model.predict(x_train)
print(classification_report(y_train, train_svm_pred))

NameError: ignored

In [ ]:
#model
dt_model = tree.DecisionTreeClassifier()
#fit
dt_model.fit(x_train, y_train)

In [ ]:
dt_pred = dt_model.predict(x_test)

In [ ]:
print(classification_report(y_test, dt_pred))

In [ ]:
train_dt_pred = dt_model.predict(x_train)
print(classification_report(y_train, train_dt_pred))

In [42]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

ValueError: ignored

In [ ]:
rf_pred = rf_model.predict(x_test)

In [ ]:
print(classification_report(y_test, rf_pred))

In [ ]:
train_rf_pred = rf_model.predict(x_train)
print(classification_report(y_train, train_rf_pred))

In [43]:
#model
xg_model = XGBClassifier()
#fit
xg_model.fit(x_train, y_train)

ValueError: ignored

In [ ]:
xg_pred = xg_model.predict(x_test)

In [ ]:
print(classification_report(y_test, xg_pred))

In [ ]:
train_xg_pred = xg_model.predict(x_train)
print(classification_report(y_train, train_xg_pred))